In [1]:
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge geopy --yes
#!pip install shapely

import shapely.geometry
import pyproj
import math

import pandas as pd # library for data analsysis
import numpy as np

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from folium import plugins
from folium.plugins import HeatMap

import pickle


## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Italian restaurant** in **Ho Chi Minh City**, Vietnam.

Since there are lots of restaurants in Ho Chi Minh City, we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using GeoPy **Nominatim API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Ho Chi Minh City center will be obtained using GeoPy **Nominatim API geocoding**

Let's first find the latitude & longitude of Ho Chi Minh City center.

In [2]:
address = 'Ho Chi Minh City'

# Define a user_agent vi_explorer.
geolocator = Nominatim(user_agent="vi_explorer")

location = geolocator.geocode(address)
loc_latitude = location.latitude
loc_longitude = location.longitude

location_center = [loc_latitude, loc_longitude]

print('Geographical coordinate of {} center are {}.'.format(address, location_center))

Geographical coordinate of Ho Chi Minh City center are [10.7758439, 106.7017555].


Let's create a grid of area candidates, equaly spaced, within ~6km around city center. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [3]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [4]:
location_center_x, location_center_y = lonlat_to_xy(loc_longitude, loc_latitude) # Location center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = location_center_x - 6000
x_step = 600
y_min = location_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(location_center_x, location_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [5]:
map_center = folium.Map(location=location_center, zoom_start=14)
folium.Marker(location_center, popup=address).add_to(map_center)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=300, color='blue', opacity=0.5, fill=False).add_to(map_center)
map_center

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from city center. 

In [6]:
def get_address(latitude, longitude):
    location = geolocator.reverse('{}, {}'.format(latitude, longitude))
    return location.address

print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    addr = get_address(lat, lon)
    addr = addr.replace(', Việt Nam', '').replace(', Thành phố Hồ Chí Minh', '') # We don't need city and country parts of address
    addresses.append(addr)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


Let's now place all this into a Pandas dataframe.

In [7]:
df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})
print(df_locations.shape)
df_locations.head(10)

(364, 6)


,Address,Latitude,Longitude,X,Y,Distance from center
0,"Lẩu Kichi Kichi, Lê Quý Đôn, Võ Thị Sáu, Phườn...",10.780574,106.691692,1.522905e+07,1.090728e+07,5992.495307
1,"Võ Thị Sáu, Phường Võ Thị Sáu, Quận 3, 7000000",10.779478,106.691541,1.522965e+07,1.090728e+07,5840.376700
2,"Phòng Khám Sản Nhi Diamond, 9, Trần Quốc Thảo,...",10.778382,106.691389,1.523025e+07,1.090728e+07,5747.173218
3,"Maritime, Võ Văn Tần, Võ Thị Sáu, Phường Võ Th...",10.777287,106.691238,1.523085e+07,1.090728e+07,5715.767665
4,"Tip top Karaoke, Trương Định, Võ Thị Sáu, Phườ...",10.776191,106.691087,1.523145e+07,1.090728e+07,5747.173218
5,"Jollibee, 204, Nguyễn Thị Minh Khai, Khu phố 1...",10.775096,106.690936,1.523205e+07,1.090728e+07,5840.376700
6,"Cty lương thực, Nguyễn Thị Minh Khai, Khu phố ...",10.774001,106.690785,1.523265e+07,1.090728e+07,5992.495307
7,"Sherwood Residence, 127, Pasteur, Võ Thị Sáu, ...",10.782088,106.692878,1.522815e+07,1.090780e+07,5855.766389
8,"war museum, Đường Nguyễn Ðình Chiểu, Võ Thị Sá...",10.780992,106.692727,1.522875e+07,1.090780e+07,5604.462508
9,"VN Q1 coffee Thức, Lê Quý Đôn, Võ Thị Sáu, Phư...",10.779896,106.692576,1.522935e+07,1.090780e+07,5408.326913


Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.

In [8]:
foursquare_client_id = '...'
foursquare_client_secret = '...'

In [9]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = addr.replace(', Việt Nam', '').replace(', Thành phố Hồ Chí Minh', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [10]:
# Let's now go over our neighborhood locations and get nearby restaurants.
# We'll also maintain a dictionary of all found restaurants and all found italian restaurants

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to make sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local files
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local files to save on Foursquare quota limit
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
       

Restaurant data loaded.


In [11]:
print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in every area with radius=300m:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 395
Total number of Italian restaurants: 19
Percentage of Italian restaurants: 4.81%
Average number of restaurants in every area with radius=300m: 13.502747252747254


Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [12]:
map_center = folium.Map(location=location_center, zoom_start=14)
folium.Marker(location_center, popup=address).add_to(map_center)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_center)
map_center

Looking good. So now we have all the restaurants in area within few kilometers from city center, and we know which ones are Italian restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of city center that have low restaurant density, particularly those with low number of Italian restaurants. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from city center**. We have also **identified Italian restaurants** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of city center - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no Italian restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create **clusters of locations that meet some basic requirements** established in discussion with stakeholders: we will take into consideration locations with **only one restaurants in radius of 250 meters**, and we want locations **without Italian restaurants in radius of 500 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [13]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"Lẩu Kichi Kichi, Lê Quý Đôn, Võ Thị Sáu, Phườn...",10.780574,106.691692,1.522905e+07,1.090728e+07,5992.495307,32
1,"Võ Thị Sáu, Phường Võ Thị Sáu, Quận 3, 7000000",10.779478,106.691541,1.522965e+07,1.090728e+07,5840.376700,29
2,"Phòng Khám Sản Nhi Diamond, 9, Trần Quốc Thảo,...",10.778382,106.691389,1.523025e+07,1.090728e+07,5747.173218,30
3,"Maritime, Võ Văn Tần, Võ Thị Sáu, Phường Võ Th...",10.777287,106.691238,1.523085e+07,1.090728e+07,5715.767665,23
4,"Tip top Karaoke, Trương Định, Võ Thị Sáu, Phườ...",10.776191,106.691087,1.523145e+07,1.090728e+07,5747.173218,19
5,"Jollibee, 204, Nguyễn Thị Minh Khai, Khu phố 1...",10.775096,106.690936,1.523205e+07,1.090728e+07,5840.376700,14
6,"Cty lương thực, Nguyễn Thị Minh Khai, Khu phố ...",10.774001,106.690785,1.523265e+07,1.090728e+07,5992.495307,13
7,"Sherwood Residence, 127, Pasteur, Võ Thị Sáu, ...",10.782088,106.692878,1.522815e+07,1.090780e+07,5855.766389,8
8,"war museum, Đường Nguyễn Ðình Chiểu, Võ Thị Sá...",10.780992,106.692727,1.522875e+07,1.090780e+07,5604.462508,30
9,"VN Q1 coffee Thức, Lê Quý Đôn, Võ Thị Sáu, Phư...",10.779896,106.692576,1.522935e+07,1.090780e+07,5408.326913,33


OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [14]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Italian restaurant
0,"Lẩu Kichi Kichi, Lê Quý Đôn, Võ Thị Sáu, Phườn...",10.780574,106.691692,1.522905e+07,1.090728e+07,5992.495307,32,1921.587500
1,"Võ Thị Sáu, Phường Võ Thị Sáu, Quận 3, 7000000",10.779478,106.691541,1.522965e+07,1.090728e+07,5840.376700,29,1339.665224
2,"Phòng Khám Sản Nhi Diamond, 9, Trần Quốc Thảo,...",10.778382,106.691389,1.523025e+07,1.090728e+07,5747.173218,30,785.434468
3,"Maritime, Võ Văn Tần, Võ Thị Sáu, Phường Võ Th...",10.777287,106.691238,1.523085e+07,1.090728e+07,5715.767665,23,398.888074
4,"Tip top Karaoke, Trương Định, Võ Thị Sáu, Phườ...",10.776191,106.691087,1.523145e+07,1.090728e+07,5747.173218,19,649.088659
5,"Jollibee, 204, Nguyễn Thị Minh Khai, Khu phố 1...",10.775096,106.690936,1.523205e+07,1.090728e+07,5840.376700,14,569.375671
6,"Cty lương thực, Nguyễn Thị Minh Khai, Khu phố ...",10.774001,106.690785,1.523265e+07,1.090728e+07,5992.495307,13,416.322103
7,"Sherwood Residence, 127, Pasteur, Võ Thị Sáu, ...",10.782088,106.692878,1.522815e+07,1.090780e+07,5855.766389,8,1946.140689
8,"war museum, Đường Nguyễn Ðình Chiểu, Võ Thị Sá...",10.780992,106.692727,1.522875e+07,1.090780e+07,5604.462508,30,1718.835689
9,"VN Q1 coffee Thức, Lê Quý Đôn, Võ Thị Sáu, Phư...",10.779896,106.692576,1.522935e+07,1.090780e+07,5408.326913,33,1685.624119


In [15]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 1671.8573940007043


OK, so **on average Italian restaurant can be found within ~1.7km** from every area center candidate.

Now let's crete a map showing **heatmap / density of restaurants**.

In [16]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]
italian_latlons = [[res[2], res[3]] for res in italian_restaurants.values()]

In [17]:
map_all = folium.Map(location=location_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_all)
HeatMap(restaurant_latlons).add_to(map_all)
folium.Marker(location_center).add_to(map_all)
map_all

Let's create another heatmap map showing **heatmap/density of Italian restaurants** only.

In [18]:
map_italian = folium.Map(location=location_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_italian)
HeatMap(italian_latlons).add_to(map_italian)
folium.Marker(location_center).add_to(map_italian)
map_italian

This map is not so 'hot' (Italian restaurants represent a subset of ~4.8% of all restaurants in city center). We will now define the region of interest, which will include low-restaurant-count parts closest to city center.

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 100m appart).

In [19]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 100
y_step = 100 * k 
roi_x_min = location_center_x
roi_y_min = location_center_y

roi_center_x = location_center_x
roi_center_y = location_center_y

roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

586 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest Italian restaurant**.

In [20]:
def count_restaurants_nearby(x, y, restaurants, radius=250):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_italian_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, italian_restaurants)
    roi_italian_distances.append(distance)
print('done.')

Generating data on location candidates... done.


In [21]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Italian restaurant':roi_italian_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Italian restaurant
0,10.775768,106.701779,1.523090e+07,1.091299e+07,0,675.351163
1,10.775585,106.701754,1.523100e+07,1.091299e+07,0,629.995284
2,10.775403,106.701728,1.523110e+07,1.091299e+07,0,598.071002
3,10.775220,106.701703,1.523120e+07,1.091299e+07,0,581.793596
4,10.775037,106.701678,1.523130e+07,1.091299e+07,0,582.476311
5,10.774855,106.701652,1.523140e+07,1.091299e+07,0,600.061262
6,10.774672,106.701627,1.523150e+07,1.091299e+07,0,633.141677
7,10.774490,106.701602,1.523160e+07,1.091299e+07,0,653.269416
8,10.774307,106.701576,1.523170e+07,1.091299e+07,0,554.084946
9,10.774125,106.701551,1.523180e+07,1.091299e+07,0,455.257428


OK. Let us now **filter** those locations: we're interested only in **locations with one restaurant in radius of 250 meters**, and **no Italian restaurants in radius of 500 meters**.

In [22]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']<=1))
print('Locations with only one restaurant nearby:', good_res_count.sum())

good_ita_distance = np.array(df_roi_locations['Distance to Italian restaurant']>=500)
print('Locations with no Italian restaurants within 500m:', good_ita_distance.sum())

good_locations = np.logical_and(good_res_count, good_ita_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]

Locations with only one restaurant nearby: 449
Locations with no Italian restaurants within 500m: 358
Locations with both conditions met: 324


Let's see how this looks on a map.

In [23]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_location = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_location)
HeatMap(restaurant_latlons).add_to(map_location)
folium.Marker(location_center).add_to(map_location)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_location) 
map_location

Looking good. We now have a bunch of locations close to city center, and we know that each of those locations has only one restaurant in radius of 250m, and no Italian restaurant closer than 500m. Any of those locations is a potential candidate for a new Italian restaurant, at least based on nearby competition.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [24]:
from sklearn.cluster import KMeans

number_of_clusters = 6

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_location = folium.Map(location=roi_center, zoom_start=16)
folium.TileLayer('cartodbpositron').add_to(map_location)
folium.Marker(location_center).add_to(map_location)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=150, color='green', fill=True, fill_opacity=0.25).add_to(map_location) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_location)
map_location

Finaly, let's **reverse geocode those candidate area centers to get the addresses** which can be presented to stakeholders.

In [25]:
candidate_area_addresses = []
print('Addresses of centers of areas recommended for further analysis:')
for lon, lat in cluster_centers:
    addr = get_address(lat, lon).replace(', Thành phố Hồ Chí Minh', '').replace(', Việt Nam', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, location_center_x, location_center_y)
    print('{}{} => {:.1f}km from {} center'.format(addr, ' '*(50-len(addr)), d/1000, address))

Addresses of centers of areas recommended for further analysis:
Chung cư 42 Tôn Thất Thiệp, 42, Tôn Thất Thiệp, Phường Bến Nghé, Quận 1, 00084 => 1.2km from Ho Chi Minh City center
Phố đi bộ Nguyễn Huệ, Nguyễn Huệ, Phường Bến Nghé, Quận 1, 71006 => 0.5km from Ho Chi Minh City center
Phố đi bộ Nguyễn Huệ, Nguyễn Huệ, Phường Bến Nghé, Quận 1, 71006 => 1.3km from Ho Chi Minh City center
Phố đi bộ Nguyễn Huệ, Nguyễn Huệ, Phường Bến Nghé, Quận 1, 71006 => 2.3km from Ho Chi Minh City center
86, Pasteur, Phường Bến Nghé, Quận 1, 00084        => 2.1km from Ho Chi Minh City center
Trung Nguyen Coffee, Đông Du, Phường Bến Nghé, Quận 1, 00084 => 1.3km from Ho Chi Minh City center


In [26]:
map_center = folium.Map(location=roi_center, zoom_start=16)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_center) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=150, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.01).add_to(map_center)
map_center

This concludes our analysis. We have created 6 addresses representing centers of zones containing locations with low number of restaurants and no Italian restaurants nearby, all zones being close to city center. Although zones are shown on map with a radius of ~150 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations.

## Results and Discussion <a name="results"></a>

Our analysis shows that although there are nearly 400 restaurants in Ho Chi Minh City center, there are pockets of low restaurant density. We first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than one restaurant in radius of 250m and those with an Italian restaurant closer than 500m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 6 zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to city center but not crowded with existing restaurants (particularly Italian) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify areas close to Ho Chi Minh City center with low number of restaurants (particularly Italian restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new Italian restaurant. By calculating restaurant density distribution from Foursquare data we generated collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location, levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.